In [4]:
import requests
from features 

In [5]:
endpoint = 'http://localhost:1769/predict'

In [9]:
resp = requests.get(endpoint, params={'player_id': 'f1ee39c5-ba84-4324-b5ce-81465d4fb76e', 
                                      'postseason_flag': 0, 
                                      'num_games': 20, 
                                      'league': 'KHL'})
resp.json()

{'assists': 5.07622764595297, 'goals': 3.372729106962169, 'season': '2020-21'}